In [14]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [15]:
df=pd.read_csv('Training_DatasetV3.csv')
df['date_GMT'] = pd.to_datetime(df['date_GMT'], format='%b %d %Y - %I:%M%p')
df

,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,home_ppg,away_ppg,Result,...,away_team_fouls,home_team_possession,away_team_possession,average_goals_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_btts_yes,odds_btts_no,stadium_name
0,2014-08-16 11:45:00,complete,NaN,Manchester United,Swansea City,Mike Dean,1,2.32,1.26,L,...,20.0,58.0,42.0,0.00,1.39,5.02,9.99,0.0,0.0,Old Trafford (Manchester)
1,2014-08-16 14:00:00,complete,NaN,Leicester City,Everton,Mike Jones,1,1.37,1.00,D,...,11.0,42.0,58.0,0.00,3.24,3.43,2.38,0.0,0.0,King Power Stadium (Leicester- Leicestershire)
2,2014-08-16 14:00:00,complete,NaN,Queens Park Rangers,Hull City,Craig Pawson,1,1.21,0.79,L,...,10.0,51.0,49.0,0.00,2.45,3.25,3.29,0.0,0.0,Kiyan Prince Foundation Stadium (London)
3,2014-08-16 14:00:00,complete,NaN,Stoke City,Aston Villa,Anthony Taylor,1,1.74,0.89,L,...,7.0,54.0,46.0,0.00,2.10,3.37,4.04,0.0,0.0,bet365 Stadium (Stoke-on-Trent- Staffordshire)
4,2014-08-16 14:00:00,complete,NaN,West Bromwich Albion,Sunderland,Neil Swarbrick,1,1.32,0.95,D,...,9.0,56.0,44.0,0.00,2.21,3.42,3.64,0.0,0.0,The Hawthorns (West Bromwich)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2024-05-19 15:00:00,incomplete,NaN,Crystal Palace,Aston Villa,NaN,38,1.14,1.67,NaN,...,NaN,NaN,NaN,3.00,0.00,0.00,0.00,0.0,0.0,Selhurst Park (London)
3796,2024-05-19 15:00:00,incomplete,NaN,Liverpool,Wolverhampton Wanderers,NaN,38,2.60,1.13,NaN,...,NaN,NaN,NaN,3.27,0.00,0.00,0.00,0.0,0.0,Anfield (Liverpool)
3797,2024-05-19 15:00:00,incomplete,NaN,Luton Town,Fulham,NaN,38,0.80,0.73,NaN,...,NaN,NaN,NaN,3.24,0.00,0.00,0.00,0.0,0.0,Kenilworth Road (Luton- Bedfordshire)
3798,2024-05-19 15:00:00,incomplete,NaN,Manchester City,West Ham United,NaN,38,2.33,1.33,NaN,...,NaN,NaN,NaN,3.40,0.00,0.00,0.00,0.0,0.0,Etihad Stadium (Manchester)


In [ ]:
df["home_team_code"] = df["home_team_name"].astype("category").cat.codes
df["away_team_code"] = df["away_team_name"].astype("category").cat.codes
df["stadium_code"] = df["stadium_name"].astype("category").cat.codes
df["day_of_week"] = df["date_GMT"].dt.dayofweek

df["target"] = df["Result"].map({"W": 1, "L": 0, "D": 2})

df

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10)
#rf= LogisticRegression()
train = df[df["date_GMT"] < '2022-08-05']
test = df[(df["date_GMT"] >= '2022-08-05') & (df["date_GMT"] < '2024-04-01')]
predictors = ["home_team_code", "away_team_code", "stadium_code", "day_of_week", "Game Week", 
              "home_ppg", "away_ppg" ]
rf.fit(train[predictors], train["target"])
preds = rf.predict(test[predictors])


In [ ]:
error = accuracy_score(test["target"], preds)
print(error)

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])


In [ ]:
future = df[df["date_GMT"] > '2024-04-01']
predictions=rf.predict(future[predictors])
future["Predictions"]=predictions
LiverPool=future[(future['home_team_name'] == 'Liverpool') | (future['away_team_name'] == 'Liverpool')]
Manchester_City	=future[(future['home_team_name'] == 'Manchester City') | (future['away_team_name'] == 'Manchester City')]
Arsenal=future[(future['home_team_name'] == 'Arsenal') | (future['away_team_name'] == 'Arsenal')]

In [ ]:
# Liverpool points
L_value_counts = LiverPool['Predictions'].value_counts()
Liverpool_Points_from_Remaining_Matches = L_value_counts.get(1, 0) * 3 + L_value_counts.get(2, 0) * 1
print("Liverpool_Points_from_Remaining_Matches =", Liverpool_Points_from_Remaining_Matches)

# Manchester City points
M_value_counts = Manchester_City['Predictions'].value_counts()
Manchester_City_Points_from_Remaining_Matches = M_value_counts.get(1, 0) * 3 + M_value_counts.get(2, 0) * 1
print("Manchester_City_Points_from_Remaining_Matches =", Manchester_City_Points_from_Remaining_Matches)

# Arsenal points
A_value_counts = Arsenal['Predictions'].value_counts()
Arsenal_Points_from_Remaining_Matches = A_value_counts.get(1, 0) * 3
print("Arsenal_Points_from_Remaining_Matches =", Arsenal_Points_from_Remaining_Matches)

In [ ]:
print("Liverpool_Points_End_of_League=",Liverpool_Points_from_Remaining_Matches + 67 )
print("Manchester_City_Points_End_of_League=", Manchester_City_Points_from_Remaining_Matches + 64 )
print("Arsenal_Points_End_of_League=",Arsenal_Points_from_Remaining_Matches + 65 )